# Scraper Post-Request

Der Reihe nach:

* Url(mit post-zusatz) aufrufen.
* Dabei Post-Info senden
* Antwort als Json weiterverarbeiten. 

For-Loop:

* Für das Array das zurück kommt eine For-Loop bauen. 
* Darin ein Dictionary aufsetzen, welches die Angaben die mich interessieren Pandas-tauglich macht 
* & Das zu einer Liste hinzufügen.
* in der For-Loop die sub_url des Eintrags aufrufen (url + id)
* Mit BeautifulSoup auf die sub_page angucken & die Elemente "legend" abgreifen
* Das legend-Element das "Herr"/"Frau" enthält zum dictionary hinzufügen. 

Die fertige Liste mit Pandas angucken und freuen. 

In [2]:
# Sachen importieren

import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# Sachen vorbereiten

data = {'page': '1',
        'pageSize': '1593',
        'mainSearchText': '*'
       }

cookies = {'ASP.NET_SessionId': 'nt4pmcae3gesz4kk41h2lang'}

url = 'http://appl.fr.ch/ate/Person/FilterGridWithIds'


# Zugriff auf die Webseite

response = requests.post(url, data=data, cookies=cookies)

response_as_json = json.loads(response.text)

response_as_json

In [ ]:
# Zugreifen auf 'Data'
response_as_json['Data']

In [ ]:
# Zugreifen auf das zweite Element von 'Data'
response_as_json['Data'][1]

So schön. Aus dem baue ich jetzt eine For-Loop mit dictionary.

In [ ]:
# For Loop


verzeichnis = []

for person in response_as_json['Data']:
    thisperson = {}
    thisperson['1 Name'] = person['DisplayName'] if not person['DisplayName'] is None else ''
    thisperson['2 ID'] = person['Id'] if not person['Id'] is None else ''
    thisperson['3 Email'] = person['Email'] if not person['Email'] is None else ''
    thisperson['4 Direktion'] = person['Roles'][0]['DirectionName'] if not person['Roles'] is None and len(person['Roles']) > 0 else ''
    thisperson['5 Sektion'] = person['UnitsDisplay'] if not person['UnitsDisplay'] is None else ''
    thisperson['6 Abteilung'] = person['UnitsExports'] if not person['UnitsExports'] is None else ''
    thisperson['7 Funktion'] = person['RolesDisplay'] if not person['RolesDisplay'] is None else ''
    
    verzeichnis.append(thisperson)
    
pd.DataFrame(verzeichnis)

### Herr / Frau Zuordnung

Jetzt wird es erst richtig interessant: in meine Post-Request-Abfrage baue ich eine Abfrage mit BeautifulSoup ein.


In [ ]:
# 1 Weil das alles sonst zu kompliziert ist, scrape ich erst einmal eine Sub-Page.


sub_url = 'http://appl.fr.ch/ate/Person/Detail/1544'

r = requests.get(sub_url)
soup = BeautifulSoup(r.content, 'lxml')

legend = soup.find_all('legend')[0].text

print(legend)
print(type(legend))

legend  # <- Das ist noch nicht schön. Weil im Dictionary brauche ich ja das Element an sich, nicht 
# was sich daraus printen lässt.

In [ ]:
# 2 Das span-Tag tilgen:

legende = soup.find_all('legend')[0]

for span in legende('span'):
    span.decompose()

legende.text
# Bizli schöner

In [ ]:
# Daraus eine Funktion bauen: 


sub_url = 'http://appl.fr.ch/ate/Person/Detail/1544'

def get_gender_attr (sub_url):
    r = requests.get(sub_url)
    soup = BeautifulSoup(r.content, 'lxml')
    legend = soup.find_all('legend')[0]
    for span in legende('span'):
        span.decompose()




## Synthesis: Scraper mit Post-Request & BeautifulSoup:

Weil das oben so toll geklappt hat kopiere ich den Code von oben hierhin, damit ich oben nichts kaputt mache


In [7]:
# Sachen vorbereiten

data = {'page': '1',
        'pageSize': '50',  # nur 50 um den Code erst zu testen
        'mainSearchText': '*'
       }

cookies = {'ASP.NET_SessionId': 'nt4pmcae3gesz4kk41h2lang'}

url = 'http://appl.fr.ch/ate/Person/FilterGridWithIds'

sub_base = 'http://appl.fr.ch/ate/Person/Detail/'

# Funktion die ich nachher brauche

def get_gender_attr (sub_url):
    r = requests.get(sub_url)
    soup = BeautifulSoup(r.content, 'lxml')
    legende = soup.find_all('legend')[0]
    for span in legende('span'):
        span.decompose()
    return legende


# Zugriff auf die Webseite

response = requests.post(url, data=data, cookies=cookies)
response_as_json = json.loads(response.text)

# For Loop

verzeichnis = []
i = 1

for person in response_as_json['Data']:
    thisperson = {}
    thisperson['1 Name'] = person['DisplayName'] if not person['DisplayName'] is None else ''
    thisperson['2 ID'] = person['Id'] if not person['Id'] is None else ''
    thisperson['3 Email'] = person['Email'] if not person['Email'] is None else ''
    thisperson['4 Direktion'] = person['Roles'][0]['DirectionName'] if not person['Roles'] is None and len(person['Roles']) > 0 else ''
    thisperson['5 Sektion'] = person['UnitsDisplay'] if not person['UnitsDisplay'] is None else ''
    thisperson['6 Abteilung'] = person['UnitsExports'] if not person['UnitsExports'] is None else ''
    thisperson['7 Funktion'] = person['RolesDisplay'] if not person['RolesDisplay'] is None else ''
    
    # Uuund jetzt das Sub-Page Aufrufen.
    get_gender_attr(sub_base + str(thisperson['2 ID']))  
    thisperson['8 Gender'] = legende
    print(i)
    i += 1
    
    
    verzeichnis.append(thisperson)
    
pd.DataFrame(verzeichnis)

NameError: name 'legende' is not defined

In [14]:
# Sachen vorbereiten

data = {'page': '1',
        'pageSize': '1593',  
        'mainSearchText': '*'
       }

cookies = {'ASP.NET_SessionId': 'nt4pmcae3gesz4kk41h2lang'}

url = 'http://appl.fr.ch/ate/Person/FilterGridWithIds'
sub_base = 'http://appl.fr.ch/ate/Person/Detail/'


# Zugriff auf die Webseite

response = requests.post(url, data=data, cookies=cookies)
response_as_json = json.loads(response.text)

# For Loop

verzeichnis = []

for person in response_as_json['Data']:
    thisperson = {}
    thisperson['1 Name'] = person['DisplayName'] if not person['DisplayName'] is None else ''
    thisperson['2 ID'] = person['Id'] if not person['Id'] is None else ''
    thisperson['3 Email'] = person['Email'] if not person['Email'] is None else ''
    thisperson['4 Direktion'] = person['Roles'][0]['DirectionName'] if not person['Roles'] is None and len(person['Roles']) > 0 else ''
    thisperson['5 Sektion'] = person['UnitsDisplay'] if not person['UnitsDisplay'] is None else ''
    thisperson['6 Abteilung'] = person['UnitsExports'] if not person['UnitsExports'] is None else ''
    thisperson['7 Funktion'] = person['RolesDisplay'] if not person['RolesDisplay'] is None else ''
    
    # Uuund das Sub-Page Aufrufen.
    sub_url = sub_base + str(person['Id'])
    
    r = requests.get(sub_url)
    soup = BeautifulSoup(r.content, 'lxml')
    legend = soup.find_all('legend')[0]
    for span in legend('span'):
        span.decompose()
    
    thisperson['8 Gender'] = legend
    
    verzeichnis.append(thisperson)
    
pd.DataFrame(verzeichnis)

,1 Name,2 ID,3 Email,4 Direktion,5 Sektion,6 Abteilung,7 Funktion,8 Gender
0,--- ---,96,,Staatskanzlei,SK - Staatsarchiv,Staatsarchiv,Sekretariat,<legend> Madame --- --- </legend>
1,Abeztout Abderrahim,4423,abderrahim.abeztout@fr.ch,"Raumplanungs-, Umwelt- und Baudirektion",RUBD - Hochbauamt,Technische Abteilung,Architektin,<legend> Monsieur Abderrahim Abeztout </...
2,Ackermann Christophe,1544,christophe.ackermann@fr.ch,Finanzdirektion,FIND - Finanzinspektorat,Finanzinspektorat,Adjunkt der Dienstchefin,<legend> Monsieur Christophe Ackermann <...
3,Ackermann Urs,3557,urs.ackermann@fr.ch,Direktion der Institutionen und der Land- und ...,ILFD - Amt für institutionelle Angelegenheiten...,Abteilung Institutionen und Rechtsangelenheiten,Höherer Verwaltungssachbearbeiter,<legend> Monsieur Urs Ackermann </legend>
4,Adam Samuel,2628,,Direktion für Gesundheit und Soziales,GSD - Kantonsarztamt,Kantonsarztamt,Adjunkt des Dienstchefs,<legend> Monsieur Samuel Adam </legend>
5,Aebischer Adrian,1289,adrian.aebischer@fr.ch,Direktion der Institutionen und der Land- und ...,ILFD - Amt für Wald und Natur,"Sektor Faune, Biodiversität, Jagd und Fischerei",Wissenschaftlicher Mitarbeiter Waldbiodiversität,<legend> Monsieur Adrian Aebischer </leg...
6,Aebischer Christoph,1580,,Volkswirtschaftsdirektion,VWD - Wirtschaftsförderung Kanton Freiburg,Wirtschaftsförderung Kanton Freiburg,Projektleiter,<legend> Monsieur Christoph Aebischer </...
7,Aebischer Fernand,36,Non renseigné,Verschiedene,Vers - Sportvereinigung des Staates Freiburg,Sportarten,Kegeln,<legend> Monsieur Fernand Aebischer </le...
8,Aebischer Florent,3328,florent.aebischer@fr.ch,Volkswirtschaftsdirektion,VWD - Handelsregisteramt,Handelsregisteramt,Sekretariat und Schalter,<legend> Monsieur Florent Aebischer </le...
9,Aebischer Jean-Marc,1038,,"Direktion für Erziehung, Kultur und Sport",EKSD - Amt für Sport,Amt für Sport,Pädagogischer Mitarbeiter,<legend> Monsieur Jean-Marc Aebischer </...


In [15]:


pd.DataFrame(verzeichnis).to_csv('Verzeichnis_brut.csv')